In [ ]:
import datetime
import random
import psycopg2
from geopy.geocoders import Nominatim
from geopy.distance import geodesic

conn=psycopg2.connect(host='host', dbname='dbname',user='user',password='password',port=port)
cur=conn.cursor()

def get_distance(address1, address2):
    geolocator = Nominatim(user_agent="my-app")  # geolocator 객체 생성
    location1 = geolocator.geocode(address1)  # 첫 번째 주소의 위치 정보 가져오기
    location2 = geolocator.geocode(address2)  # 두 번째 주소의 위치 정보 가져오기
    if location1 is None or location2 is None:
        return None
    point1 = (location1.latitude, location1.longitude)  # 첫 번째 주소의 위도와 경도
    point2 = (location2.latitude, location2.longitude)  # 두 번째 주소의 위도와 경도
    distance = geodesic(point1, point2).meters  # 거리 계산 (킬로미터 단위로 반환)
    return distance

def search():
    print("****병원 찾기****")
    region=input("지역을 입력하세요 : ")
    name=input("시설 이름을 선택하세요 : ")
    cur.execute("select f.address from DBPROJECT.FACILITY F inner join DBPROJECT.REGION R on F.REGION_ID =R.REGION_ID where R.REGION_NAME ='%s' and F.facility_name='%s'; "%(region,name))
    facility_address=cur.fetchall() #시설 주소값 저장
    facility_address=facility_address[0][0]
    cur.execute("SELECT h.address,h.hospital_name from DBPROJECT.REGION R inner join DBPROJECT.HOSPITAL H on r.REGION_ID =h.REGION_ID where r.REGION_name ='%s'and h.hospital_type not like '%%응급%%' ;"%(region))
    hospitals=cur.fetchall() #검색한 지역 일반병원들 주소값 저장
    cur.execute("SELECT h.address,h.hospital_name from DBPROJECT.REGION R inner join DBPROJECT.HOSPITAL H on r.REGION_ID =h.REGION_ID where r.REGION_name ='%s'and h.hospital_type like '%%응급%%' ;"%(region))
    hospitals1=cur.fetchall()
    min_distance=9999
    count=1
    for first, last in hospitals:
        #if(count==10): break
        #count+=1
        now = get_distance(facility_address, first)
        if now is not None:
            min_distance = min(now, min_distance)
            if now == min_distance:
                min_hospital = last
    min_distance1=9999
    count=1
    for first, last in hospitals1:
        #if(count==10): break
        #count+=1
        now = get_distance(facility_address, first)
        if now is not None:
            min_distance1 = min(now, min_distance1)
            if now == min_distance1:
                min_hospital1 = last

    print("가장 가까운 병원은 %s 이고 거리는 약 %d 미터입니다"%(min_hospital,min_distance))
    print("가장 가까운 응급병원은 %s 이고 거리는 약 %d 미터입니다"%(min_hospital1,min_distance1))
search()
